In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.neighbors import KNeighborsRegressor

In [2]:
oppScrData = pd.read_excel(r'sample_data/OppScrData.xlsx')

In [30]:
def preprocess_clinical_data(oppScrData, mean=True):
    clinical_data = oppScrData.filter(['Record ID','BMI','BMI >30', 'Clinical F/U interval  [d from CT]', 'Sex', 'Tobacco', 'Met Sx', 'FRAX 10y Fx Prob (Orange-w/ DXA)',
                                'FRAX 10y Hip Fx Prob (Orange-w/ DXA)','FRS 10-year risk (%)','Age at CT' ], axis=1)
    # Replace all _,X,blanks with nan
    clinical_data = clinical_data.replace(r'_', np.nan, regex=True)
    clinical_data = clinical_data.replace(r'X', np.nan, regex=True)
    clinical_data = clinical_data.replace(r'^\s*$', np.nan, regex=True)

    # Fill na in bmi column with mean
    clinical_data['BMI'].fillna(value=clinical_data['BMI'].mean(skipna=True), inplace=True)
    
    # Fill na in bmi>30 column based on bmi col
    clinical_data.loc[clinical_data.BMI>30, 'BMI >30'] = 1
    clinical_data.loc[clinical_data.BMI<=30, 'BMI >30'] = -1
    
    clinical_data['Sex'] = np.where(clinical_data['Sex']=='Male',1,-1)
    clinical_data['Met Sx'] = np.where(clinical_data['Met Sx']=='Y',1,-1) 

    # Treat no data in tobacco as no tobacco usage 
    clinical_data['Tobacco'] = np.where(clinical_data['Tobacco']=='Yes',1,-1) 
  
    clinical_data['FRS 10-year risk (%)'] = clinical_data['FRS 10-year risk (%)'].replace("<1", 0.01, regex=True)
    clinical_data['FRS 10-year risk (%)'] = clinical_data['FRS 10-year risk (%)'].replace(">30", 0.30, regex=True)
    clinical_data['FRS 10-year risk (%)'] =  clinical_data['FRS 10-year risk (%)'] * 100
   
    cols_to_be_filled = ['FRAX 10y Fx Prob (Orange-w/ DXA)','FRAX 10y Hip Fx Prob (Orange-w/ DXA)','FRS 10-year risk (%)']
    for c in cols_to_be_filled:
      if mean:  
        clinical_data[c].fillna(value=clinical_data[c].mean(skipna=True), inplace=True)
      else :
        clinical_data[c].fillna(value=clinical_data[c].median(skipna=True), inplace=True)
    
    return clinical_data

In [43]:
def clean_ct_data(oppScrData):
    # Delete rows with empty values
    cols = ["Record ID", "L1_HU_BMD", "TAT Area (cm2)", 'Total Body                Area EA (cm2)',
       'VAT Area (cm2)', 'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)','DEATH [d from CT]']
    ct_data= oppScrData[cols]
    n = ct_data.shape[0]
    preprocessed_ct_data = pd.DataFrame( columns=cols)
    for i in range(n):
        row = ct_data.loc[i]
        ignore = False
        for k,j in enumerate(row):
          if pd.isna(j) or j == ' ': 
            if k == 12:              
              ignore = False
            else:
              ignore = True
              break
        if not ignore:
          preprocessed_ct_data.loc[i]= row
    return preprocessed_ct_data

In [37]:
clinical_data = preprocess_clinical_data(oppScrData)
ct_data = clean_ct_data(oppScrData)

In [ ]:
combined_data = clinical_data.merge(ct_data, on='Record ID', how='inner' )
combined_data.drop(columns=['Record ID'], inplace=True)
combined_data.drop(columns=['Clinical F/U interval  [d from CT]'], inplace=True)

In [48]:
print(combined_data.columns)

Index(['BMI', 'BMI >30', 'Sex', 'Tobacco', 'Met Sx',
       'FRAX 10y Fx Prob (Orange-w/ DXA)',
       'FRAX 10y Hip Fx Prob (Orange-w/ DXA)', 'FRS 10-year risk (%)',
       'Age at CT', 'L1_HU_BMD', 'TAT Area (cm2)',
       'Total Body                Area EA (cm2)', 'VAT Area (cm2)',
       'SAT Area (cm2)', 'VAT/SAT     Ratio', 'Muscle HU',
       ' Muscle Area (cm2)', 'L3 SMI (cm2/m2)', 'AoCa        Agatston',
       'Liver HU    (Median)', 'DEATH [d from CT]'],
      dtype='object')


In [49]:
clinical_data = combined_data

**Model training**

In [ ]:
sc = StandardScaler()

non_null_death_rows = clinical_data[~clinical_data['DEATH [d from CT]'].isnull()]

# Convert to years
non_null_death_rows['DEATH [d from CT]'] /= 365

y_idx = clinical_data.columns.get_loc("DEATH [d from CT]")

X = non_null_death_rows.iloc[:, 0:y_idx]
y = non_null_death_rows.iloc[:, y_idx]
X = pd.DataFrame(sc.fit_transform(X), columns = X.columns)

null_death_rows=clinical_data[clinical_data['DEATH [d from CT]'].isnull()]
X2 = null_death_rows.iloc[:, 0:y_idx]
X2 = pd.DataFrame(sc.fit_transform(X2), columns = X2.columns)

In [71]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2)

In [72]:
RFparams2={'n_estimators':[50],
          'min_samples_split':[16],
          'min_samples_leaf': [4],
           'max_features': [6],
           'max_depth': [20],
           'bootstrap': [True]}

rf = RandomForestRegressor(random_state = 21)
clf = GridSearchCV(rf, RFparams2, refit=True)
clf.fit(X_train, y_train)
clf.best_estimator_.score(X_test, y_test)

0.4709595919338694

In [73]:
p = clf.predict(X_test)
Mse = mean_squared_error(y_test, p)
error = mean_absolute_error(y_test, p)
mape = mean_absolute_percentage_error(y_test, p)
print("RMSE : %.2f" % (Mse**(1/2.0)))
print("MApe : %.2f" %(mape))
print("MAE :", error)

RMSE : 0.92
MApe : 0.23
MAE : 0.45203036757170134


**Fill death column**

In [74]:
null_death_rows['DEATH [d from CT]'] = clf.predict(X2)
df = pd.concat([non_null_death_rows,null_death_rows])
y_idx = clinical_data.columns.get_loc("DEATH [d from CT]")
y = df.iloc[:,y_idx]
X = df.iloc[:, 0:y_idx]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [69]:
y

0       0.983562
2       6.441096
3       1.802740
5       5.397260
13      0.304110
          ...   
8873    5.725122
8874    5.244797
8875    6.298674
8876    5.962178
8877    4.238722
Name: DEATH [d from CT], Length: 8878, dtype: float64

**Train model**

In [79]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2)

knn = KNeighborsRegressor(n_neighbors=9)
param_grid2 = dict(weights = ["uniform", "distance"])
    
grid = GridSearchCV(knn, param_grid2)
grid.fit(X_train, y_train)
p3 = grid.predict(X_test) 
Mse = mean_squared_error(y_test, p3)

mape=mean_absolute_percentage_error(y_test, p3)
print("RMSE: %.2f" % (Mse**(1/2.0)))
print("MApe: %.2f" %(mape))
print("MAE: %.2f" %(mean_absolute_error(y_test, p3)))


RMSE: 1.01
MApe: 0.11
MAE: 0.34
